In [1]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


XOR 연산은 입력 데이터를 X라고 할 경우 [[0, 0], [0, 1], [1, 0], [1, 1]]과 같은 경우의 수가 있다.
총 4개의 데이터가 각각 2개의 특성을 가지고 있으므로 X는 [4, 2]의 형태로 정의할 수 있다.
입력값에 따른 출력값 Y는 [[0], [1], [1], [0]]이므로 Y는 [4, 1]의 형태로 정의할 수 있다.

In [2]:
X = tf.placeholder(dtype=tf.float32, shape=[4, 2])  # 입력값
Y = tf.placeholder(dtype=tf.float32, shape=[4, 1])  # 출력값

1번째 히든 레이어에서 2개의 입력(X)과 1개의 편향값(B1)을 받아서 2개의 시그모이드 출력(Z)을 다음 레이어로 전달한다.

In [3]:
# 2개의 입력을 받는 2개의 뉴런을 만든다.
# random_normal()은 정규 분포로 부터 난수값을 반환하고 random_uniform()은 균등 분포로 부터의 난수값을 반환한다.
W1 = tf.Variable(tf.random_uniform([2, 2]))
# 각 뉴런은 1개의 편향값을 가진다.
# zeros()는 모든 원소의 값이 0인 텐서를 생성한다.
B1 = tf.Variable(tf.zeros([2]))
# 시그모이드를 거쳐 출력값으로 Z를 리턴한다. => sigmoid(W1 * X + B1)
Z = tf.sigmoid(tf.matmul(X, W1) + B1)

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W1))
print(sess.run(B1))
train_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
print(sess.run(Z, feed_dict={X: train_X}))
sess.close()

Metal device set to: Apple M2


2023-03-03 09:35:29.559796: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-03 09:35:29.560847: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-03 09:35:29.569256: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-03 09:35:29.577145: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-03 09:35:29.584358: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


[[0.62341976 0.22441173]
 [0.15846539 0.24724305]]
[0. 0.]
[[0.5        0.5       ]
 [0.5395337  0.56149787]
 [0.65099597 0.5558687 ]
 [0.6860863  0.61577535]]


2023-03-03 09:35:29.918517: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:35:29.920081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:35:29.921383: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2번째 히든 레이어에서 1번째 히든 레이어의 출력값인 Z와 1개의 편향값(B2)을 받아서 1개의 시그모이드를 출력(Y_hat)한다.

In [5]:
# Z를 입력값으로 받는 1개의 뉴런을 만든다.
W2 = tf.Variable(tf.random_uniform([2, 1]))
# 각 뉴런은 1개의 편향값을 가진다.
B2 = tf.Variable(tf.zeros([1]))
# 시그모이드를 거쳐 출력값으로 Y_hat을 리턴한다. => sigmoid(W2 * Z + B1)
Y_hat = tf.sigmoid(tf.matmul(Z, W2) + B2)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W2))
print(sess.run(B2))
print(sess.run(Y_hat, feed_dict={X: train_X}))
sess.close()

[[0.9230473 ]
 [0.24017632]]
[0.]
[[0.6414382]
 [0.6554442]
 [0.6902898]
 [0.701141 ]]


2023-03-03 09:35:50.601915: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-03 09:35:50.601942: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-03 09:35:50.605536: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:35:50.620063: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:35:50.621921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:35:50.62

손실 함수로 크로스 엔트로피를 사용하고 경사 하강법으로 모델의 매개 변수(가중치, 편향)을 최적화 한다.

In [7]:
# 크로스 엔트로피
loss = tf.reduce_mean(
    -1 * (Y * tf.log(Y_hat) + (1 - Y) * tf.log(1.0 - Y_hat))
)
# 경사 하강법
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
# 모델을 학습하기 위한 데이터를 만든다.
train_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
train_Y = [[0], [1], [1], [0]]

학습을 진행한다.

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('학습 데이터: {}'.format(train_X))
    print('=' * 80)

    for epoch in range(20001):
        sess.run(train_step, feed_dict={X: train_X, Y: train_Y})
        if epoch % 5000 == 0:
            print('epoch: {}'.format(epoch))
            print('output\n', sess.run(Y_hat, feed_dict={X: train_X, Y: train_Y}), sep='')
            print('=' * 80)

2023-03-03 09:36:17.148673: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-03 09:36:17.148700: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-03 09:36:17.152928: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:36:17.166259: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


학습 데이터: [[0, 0], [0, 1], [1, 0], [1, 1]]
epoch: 0
output
[[0.59652543]
 [0.6137105 ]
 [0.62945896]
 [0.64283615]]


2023-03-03 09:36:17.563140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


epoch: 5000
output
[[0.37980846]
 [0.5370688 ]
 [0.53975415]
 [0.58092946]]
epoch: 10000
output
[[0.11299863]
 [0.7878277 ]
 [0.7864633 ]
 [0.32995075]]
epoch: 15000
output
[[0.0407591 ]
 [0.9624168 ]
 [0.9624548 ]
 [0.04536924]]
epoch: 20000
output
[[0.02290986]
 [0.98129886]
 [0.9813136 ]
 [0.021178  ]]


학습 결과를 통해 [0, 0], [1, 1]은 0에 상당히 가까운 값을 출력하고 [0, 1], [1, 0]은 1에 상당히 가까운 값을 출력하는 것을 확인할 수 있다.